In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import os
import PIL

In [9]:
data_transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        ])

